# TransformerLense

In [70]:
import torch
import tqdm
import transformer_lens

# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("gpt2-medium")
model.tokenizer.pad_token = model.tokenizer.eos_token

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-medium into HookedTransformer


In [72]:
from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [71]:
# Run the model and get logits and activations
tokens = model.to_tokens("Hello World")
logits, activs = model.run_with_cache(tokens)

In [104]:
def mlp_out_hook(mlp_out_new, hook: HookPoint, old_activs: torch.Tensor, token_idx:int): 
    print(f'{hook.name}')
    mlp_out_old = old_activs[hook.name]
    mlp_out_new[:, token_idx, :] = mlp_out_old[:, token_idx, :]
    print("\n")
    return mlp_out_new


def intervene(new_prompt, old_activs, tok_idx = -1, l_start_end=[0, 99]):
    for layer in range(l_start_end[0], l_start_end[1]):
        if layer < model.cfg.n_layers:
            temp_hook_fn = partial(mlp_out_hook, old_activs=old_activs, token_idx=tok_idx)            
            patched_scores = model.run_with_hooks(new_tokens, fwd_hooks=[(utils.get_act_name("mlp_out", layer), temp_hook_fn)])
            print(patched_scores[:,tok_idx,:])

new_prompt = "This is great right?"
new_tokens = model.to_tokens(new_prompt)

intervene(new_tokens, activs, tok_idx = -1, l_start_end=[15, 18])

blocks.15.hook_mlp_out


tensor([[ 8.6360,  5.9398,  5.7453,  ..., -6.0667, -4.5309,  9.9179]],
       device='mps:0', grad_fn=<SliceBackward0>)
blocks.16.hook_mlp_out


tensor([[ 8.3343,  5.7695,  5.0471,  ..., -5.8012, -4.0417, 10.2291]],
       device='mps:0', grad_fn=<SliceBackward0>)
blocks.17.hook_mlp_out


tensor([[ 8.6266,  5.8475,  5.3887,  ..., -5.5492, -4.3809, 10.1597]],
       device='mps:0', grad_fn=<SliceBackward0>)
